# 04) Modeling

Here you will find the different models built in order to attempt predicting taxi fares. 
They include: 
- 4.01 GradientBoost (winner)
- 4.02 RandomForest
- 4.03 LinearRegression
- 4.04 XGBoost 

## 4.01 GradientBoost

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

pd.set_option('display.max_columns', None)

In [38]:
#In order to run the model faster, I had to sample it down to 700,000 instead of nearly 2M
new_df = pd.read_csv('../data/clean/041324_taxi_recs.csv')
new_df = new_df.set_index('req_index')
new_df = new_df.sample(n=700000, random_state=2024)
print(new_df.shape)
new_df.head()

(700000, 14)


,trip_miles,tips,congestion_surcharge,temp,preciptype,zone,borough_name,trip_duration,month,day_of_month,driver_made,day_of_week,hour,minute
req_index,,,,,,,,,,,,,,
2022-10-19,2.38,5.0,0.00,48.9,0,Saint George/New Brighton,Staten Island,9.0,10,19,14.82,Wednesday,14,27
2022-07-15,10.92,0.0,0.00,78.3,0,Old Astoria,Queens,35.0,7,15,30.91,Friday,22,24
2022-03-09,1.26,0.0,0.00,37.6,3,Flushing,Queens,6.0,3,9,6.32,Wednesday,15,51
2022-12-03,1.53,0.0,2.75,52.7,1,Upper East Side North,Manhattan,7.0,12,3,5.88,Saturday,20,56
2022-11-20,0.70,0.0,0.00,34.9,2,Greenpoint,Brooklyn,3.0,11,20,6.15,Sunday,10,18


In [44]:
new_df.dtypes

trip_miles              float64
tips                    float64
congestion_surcharge    float64
temp                    float64
preciptype                int64
zone                     object
borough_name             object
trip_duration           float64
month                     int64
day_of_month              int64
driver_made             float64
day_of_week              object
hour                      int64
minute                    int64
dtype: object

In [39]:
new_df = new_df.sort_index()
new_df.shape

(700000, 14)

**Note**: By sampling down the data using .sample(), one runs the risk of the training data not including all zones in the testing data. Should this happen, tune the .sample() size and pull again.

In [40]:
#Attempthing Gradient Boost

#setting X, y
X = new_df.drop(columns=['driver_made', 'tips',	'congestion_surcharge'])
y = new_df['driver_made']

#tts
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2024)

# Instantiating
gb = GradientBoostingRegressor(random_state=2024)
ohe = OneHotEncoder()

cat_var = ['borough_name', 'zone', 'day_of_week']
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), cat_var)], remainder ='passthrough')

pipe = Pipeline([('ct', ct), ('gb', gb)])

#Fitting
pipe.fit(X_train, y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe', OneHotEncoder(),
                                                  ['borough_name', 'zone',
                                                   'day_of_week'])])),
                ('gb', GradientBoostingRegressor(random_state=2024))])

In [41]:
X_test.shape

(175000, 11)

In [42]:
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.8696095556417843, 0.8695550482254655)

In [79]:
with open('../Taxis/taxi_model.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [90]:
#Feature engineering to tryand get a better R2

#Reading in df
df = pd.read_csv('./data/nyc_taxi_records.csv')

#Setting index
df = df.set_index('req_index')
df.head()

(3975858, 21)


,PULocationID,DOLocationID,trip_miles,base_passenger_fare,tips,driver_pay,congestion_surcharge,req_time,req_date,pickup_time,dropoff_date,dropoff_time,temp,preciptype,zone,borough,tips_driver_pay,trip_duration,month,day,borough_name
req_index,,,,,,,,,,,,,,,,,,,,,
2022-01-04,183.0,20.0,4.148,17.69,4.95,12.13,0.00,07:32:25,2022-01-04,07:38:00,2022-01-04,07:53:00,27.6,0,Pelham Bay,1,17.08,900,1,4,Bronx
2022-01-04,262.0,238.0,1.630,10.11,0.00,7.68,2.75,07:15:54,2022-01-04,07:17:00,2022-01-04,07:29:00,27.6,0,Yorkville East,2,7.68,720,1,4,Manhattan
2022-01-04,50.0,161.0,1.751,10.05,2.00,7.85,2.75,07:15:54,2022-01-04,07:23:00,2022-01-04,07:32:00,27.6,0,Clinton West,2,9.85,540,1,4,Manhattan
2022-01-04,124.0,124.0,1.633,10.76,0.00,5.47,0.00,07:15:54,2022-01-04,07:23:00,2022-01-04,07:28:00,27.6,0,Howard Beach,3,5.47,300,1,4,Queens
2022-01-04,180.0,63.0,6.070,24.41,0.00,16.33,0.00,07:15:54,2022-01-04,07:19:00,2022-01-04,07:38:00,27.6,0,Ozone Park,3,16.33,1140,1,4,Queens


In [91]:
#Dropping what I willnot be using
df = df.drop(columns=['PULocationID', 'DOLocationID', 'base_passenger_fare', 'tips', 'driver_pay', 'pickup_time', 'dropoff_date', 'borough'])

#Creating new columns:

#Day of week
df['day_of_week'] = pd.to_datetime(df['req_date']).dt.strftime("%A")

#Hour of day
df['hour'] = pd.to_datetime(df['req_time']).dt.hour

#Minute of hour
df['minute'] = pd.to_datetime(df['req_time']).dt.minute

#Transforming trip duration into minutes
df['trip_duration'] = [i/60 for i in df['trip_duration']]

#Dropping the columns I no longer need
df = df.drop(columns=['req_time', 'req_date', 'dropoff_time'])
df = df.drop(columns=['req_time', 'req_date', 'dropoff_time'])

#Renaming columns to match my streamlit app
df = df.rename(columns={'tips_driver_pay': 'driver_made', 'day': 'day_of_month'})

#Checking
df.head()

,trip_miles,congestion_surcharge,req_time,req_date,dropoff_time,temp,preciptype,zone,tips_driver_pay,trip_duration,month,day,borough_name
req_index,,,,,,,,,,,,,
2022-01-04,4.148,0.00,07:32:25,2022-01-04,07:53:00,27.6,0,Pelham Bay,17.08,900,1,4,Bronx
2022-01-04,1.630,2.75,07:15:54,2022-01-04,07:29:00,27.6,0,Yorkville East,7.68,720,1,4,Manhattan
2022-01-04,1.751,2.75,07:15:54,2022-01-04,07:32:00,27.6,0,Clinton West,9.85,540,1,4,Manhattan
2022-01-04,1.633,0.00,07:15:54,2022-01-04,07:28:00,27.6,0,Howard Beach,5.47,300,1,4,Queens
2022-01-04,6.070,0.00,07:15:54,2022-01-04,07:38:00,27.6,0,Ozone Park,16.33,1140,1,4,Queens


In [114]:
#Saving clean dataframe
df.to_csv('./data/taxi_clean.csv')

In [108]:
#Attempthing Gradient Boost with more data

#Setting X2, y2
X2 = df.drop(columns=['driver_made'])
y2 = df['driver_made']

#Train Test Split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=2024)

#Instantiating GradientBoost and Preprocessor
gb = GradientBoostingRegressor(random_state=2024)
ohe = OneHotEncoder()

#Creating my column transformer
cat_var = ['borough_name', 'zone', 'day_of_week']
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), cat_var)], remainder ='passthrough')

#Creating my pipeline
pipe2 = Pipeline([('ct', ct), ('gb', gb)])

#Fitting
pipe2.fit(X2_train, y2_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe', OneHotEncoder(),
                                                  ['borough_name', 'zone',
                                                   'day_of_week'])])),
                ('gb', GradientBoostingRegressor(random_state=2024))])

In [109]:
print(X2_train.shape)
print(X2_test.shape)
print(y2_train.shape)
print(y2_test.shape)

(2981893, 12)
(993965, 12)
(2981893,)
(993965,)


In [110]:
#Getting the scores
pipe2.score(X2_train, y2_train), pipe.score(X2_test, y2_test)

(0.8736653666624892, 0.8744832863952346)

In [111]:
#Saving the model
with open('../Taxis/larger_model.pkl', 'wb') as a:
    pickle.dump(pipe2, a)

## 4.02 RandomForest

In [ ]:
#Importing the models
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

#Reading in fresh dataframe
df = pd.read_csv('../data/nyc_taxi_records.csv')

#Setting the req_index column as the index
df.set_index('req_index', inplace=True)

#Converting the index to datetime
df.index = pd.to_datetime(df.index, format= '%Y-%m-%d')

#Checking the dtypes
df.dtypes

In [ ]:
#Feature Engineering/ More EDA

#Dropping df columns I don't want
new_df = df.drop(columns=['PULocationID', 'DOLocationID',
       'pickup_time', 'dropoff_date', 'dropoff_time', 'preciptype',
      'borough', 'tips_driver_pay', 'month', 'day'])

#Creating new datetime columns
new_df['req_time'] = pd.to_datetime(new_df['req_time'], format = 'mixed')

#Hour
new_df['hour'] = new_df['req_time'].dt.hour

#Day of week
new_df['day_of_week'] = new_df['req_date'].dt.day_name()

#Dropping columns no longer need
new_df = new_df.drop(columns = ['req_time', 'req_date'])

#Dropping NAs
new_df = new_df.dropna()

#Creating a driver_made column
new_df['driver_made'] = new_df['driver_pay'] + new_df['tips']

#Dropping the columns no longer needed
new_df = new_df.drop(columns=['driver_pay', 'tips'])

In [ ]:
#Creating a smaller dataset to make it quicker
new_df = new_df.sample(n=1000000)

In [ ]:
#Make X, y
X = new_df.drop(columns=['driver_made', 'congestion_surcharge', 'base_passenger_fare'])
y = new_df['driver_made']

#TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2024)

#Instantiating
ohe = OneHotEncoder()
sc = StandardScaler()
rf = RandomForestRegressor()

cat_var = ['borough_name', 'zone', 'day_of_week']

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), cat_var)], remainder ='passthrough')

#Building the pipeline
pipe = Pipeline([
    ('ct', ct),
    ('sc', StandardScaler(with_mean=False)),
    ('rf', RandomForestRegressor(n_estimators=200, 
                                 max_depth=30, min_samples_split=.05,
                                max_features='sqrt', n_jobs=4))
     ])

#Fitting the model
pipe.fit(X_train, y_train)

#Looking at r2 scores
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

In [ ]:
#Feature engineering/tuning to build pipe2
pipe2 = Pipeline([
    ('ct', ct),
    ('sc', StandardScaler(with_mean=False)),
    ('rf', RandomForestRegressor(n_estimators=250, 
                                 max_depth=30, min_samples_split=300,
                                 max_features='sqrt', n_jobs=4))
     ])

#Fitting
pipe2.fit(X_train, y_train)

#Looking at r2 scores
pipe2.score(X_train, y_train), pipe2.score(X_test, y_test)

## 4.03 LinearRegression

In [ ]:
#Imports I'll need
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn import metrics

In [ ]:
#Reading in fresh dataframe
df = pd.read_csv('../data/nyc_taxi_records.csv')

#Setting the req_index column as the index
df.set_index('req_index', inplace=True)

#Converting the index to datetime
df.index = pd.to_datetime(df.index, format= '%Y-%m-%d')

#Checking the dtypes
df.dtypes

In [ ]:
#Feature Engineering/ More EDA

#Dropping df columns I don't want
new_df = df.drop(columns=['PULocationID', 'DOLocationID',
       'pickup_time', 'dropoff_date', 'dropoff_time', 'preciptype',
      'borough', 'tips_driver_pay', 'month', 'day'])

#Creating new datetime columns
new_df['req_time'] = pd.to_datetime(new_df['req_time'], format = 'mixed')

#Hour
new_df['hour'] = new_df['req_time'].dt.hour

#Day of week
new_df['day_of_week'] = new_df['req_date'].dt.day_name()

#Dropping columns no longer need
new_df = new_df.drop(columns = ['req_time', 'req_date'])

#Dropping NAs
new_df = new_df.dropna()

#Creating a driver_made column
new_df['driver_made'] = new_df['driver_pay'] + new_df['tips']

#Dropping the columns no longer needed
new_df = new_df.drop(columns=['driver_pay', 'tips'])

In [ ]:
#Creating a smaller dataset
new_df = new_df.sample(n=500000, random_state=2024)

In [ ]:
#Ordinal mapping to get numerics

#Days of week
days = {'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}
new_df['day_of_week'] = new_df['day_of_week'].map(days)

#Boroughs
boroughs = {'Manhattan': 1, 'Brooklyn': 2, 'Queens': 3, 'Bronx': 4, 'Staten Island': 5}
new_df['borough_name'] = new_df['borough_name'].map(boroughs)

In [ ]:
#Dummifying taxi zones
new_df = pd.get_dummies(columns=['zone'], data=new_df).astype(int)

In [ ]:
#Setting my X, y
X = new_df.drop(columns=['driver_made', 'base_passenger_fare', 'congestion_surcharge'])
y = new_df['driver_made']

#Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2024)

#Checking the baseline
y_train.mean()

In [ ]:
#Instantiating preprocessor and model
sc = StandardScaler()
lr = LinearRegression()

#Building the pipe
pipe = Pipeline([
    ('sc', StandardScaler()),
    ('lr', LinearRegression())
])

#Fitting the pipe
pipe.fit(X_train, y_train)

#Testing the pipe
pipe.score(X_test, y_test), pipe.score(X_test, y_test)

In [ ]:
# More Feature Engineering/ building a pipe with less features

#Creating new X with less features
X2 = new_df.drop(columns=['driver_made', 'congestion_surcharge', 'base_passenger_fare', 'trip_duration'])
y= new_df['driver_made']

#Perfomring tts
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, random_state=2024)


#Fitting the new tts
pipe.fit(X2_train, y_train)

#Getting Score
pipe.score(X2_test, y_test), pipe.score(X2_test, y_test)


In [ ]:
# Trying a LassoCV

# Set up a list of Lasso alphas to check.
l_alphas = np.logspace(-3, 0, 100)

# Cross-validate over our list of Lasso alphas.
lasso_cv = LassoCV(alphas=l_alphas, cv=5, max_iter=10)

sc = StandardScaler()

pipe2 = Pipeline([
    ('sc', StandardScaler()),
    ('lasso_cv', lasso_cv)
])

# Fit model using best ridge alpha!
pipe2.fit(X_train, y_train)

#Checking the scores
pipe2.score(X_test, y_test), pipe2.score(X_test, y_test)

In [ ]:
#Trying a RidgeRegression

#Setting alphas
r_alphas = np.logspace(0, 5, 100)

# Cross-validate over our list of ridge alphas.
ridge_cv = RidgeCV(alphas=r_alphas, scoring='r2', cv=5)

sc = StandardScaler()

pipe3 = Pipeline([
    ('sc', StandardScaler()),
    ('ridge_cv', ridge_cv)
])

# Fit model using best ridge alpha!
pipe3.fit(X_train, y_train)

#Getting score
pipe3.score(X_test, y_test), pipe3.score(X_test, y_test)

In [ ]:
#Checking the best alpha_
ridge_cv.alpha_

## 4.04 XGBoost

In [ ]:
#Importing model
import xgboost as xgb
from xgboost.sklearn import XGBRegressor



In [ ]:
#Importing data set
new_df = pd.read_csv('../data/taxi_vars.csv')

#Setting the index
new_df = new_df.set_index('req_index')

#Getting a smaller sample to make my models faster
new_df = new_df.sample(n=500000, random_state=2024)

#dropping unnecessary columns
new_df = new_df.drop(columns=['req_time',	'req_date'])

#Checking
new_df.head()

In [ ]:
#Making X, y
X = new_df.drop(columns=['driver_made', 'congestion_surcharge', 'base_passenger_fare'])
y = new_df['driver_made']

#Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2024)

#Instantiating preprocessor
ohe = OneHotEncoder() 

#Creating pipeline and ColumnTransformer for preprocessing
cat_var = ['borough_name', 'zone', 'day_of_week']
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

ct = ColumnTransformer([
    ('cat_pipe', cat_pipe, cat_var)])

#Instantiating XGBoost
xgb = XGBRegressor(n_estimators=250, 
                   max_depth=30, min_samples_split=200,
                   max_features=['sqrt', 'log2'], n_jobs=4, random_state=2024, 
                   enable_categorical=True)

#Building the pipeline of transformers and the model
pipe = Pipeline([
    ('ct', ct),
    ('xgb', xgb)
])

#Fitting the model
pipe.fit(X_train, y_train)

In [ ]:
#Checking the r2 scores
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

In [ ]:
#Checking RMSE scores

#Predictions
train_preds = pipe.predict(X_train)
test_preds = pipe.predict(X_test)

#Evaluate the model
train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))

#Printing scores
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

In [ ]:
#Feature engineering/tuning parameter for XGB2 model

#Instantiatingnew model and params
xgb2 = XGBRegressor(n_estimators=300, 
                   max_depth=20, min_samples_split=200,
                   max_features=['sqrt'], n_jobs=4, random_state=2024, 
                   enable_categorical=True)

#Building a second pipeline
pipe2 = Pipeline([
    ('ct', ct),
    ('xgb2', xgb2)
])

#Fitting to new pipe
pipe2.fit(X_train, y_train)

In [ ]:
#Checking XGB2 RMSE scores

#Predictions
train_preds = pipe2.predict(X_train)
test_preds = pipe2.predict(X_test)

# Evaluate the model
train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

#r2 scores
print(pipe2.score(X_train, y_train), pipe2.score(X_test, y_test))

In [ ]:
#Feature engineering/tuning parameter for XGB3/pipe3 model

#Instantiating XGB3 with new params
xgb3 = XGBRegressor(n_estimators=500, 
                   max_depth=10, min_samples_split=200,
                   min_child_weight=1, random_state=2024, 
                   enable_categorical=True)

#Building pipe3
pipe3 = Pipeline([
    ('ct', ct),
    ('xgb3', xgb3)
])

#Fitting to pipe3
pipe3.fit(X_train, y_train)

In [ ]:
#Checking XGB2 RMSE scores

#Predictions
train_preds = pipe3.predict(X_train)
test_preds = pipe3.predict(X_test)

# Evaluate the model
train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

#r2 scores
print(pipe3.score(X_train, y_train), pipe3.score(X_test, y_test))